# 🧭 #30DayMapChallenge 2024 - Day 11: Why Not Antarctica?

## Unveiling the Frozen Continent: Mapping Antarctica's Research Stations 🇦🇶

### 1. Understanding Antarctica's Unique Landscape 🗺️

Unlike the well-known Arctic region, Antarctica is often overlooked, yet it holds immense scientific and geopolitical significance. As the southernmost continent on Earth, Antarctica is a vast, icy landmass that covers over 14 million square kilometers, making it the fifth-largest continent.

The Antarctic continent is governed by the Antarctic Treaty System (ATS), which was established in 1959 to ensure the continent remains a peaceful and collaborative research hub. The ATS is overseen by the Scientific Committee on Antarctic Research (SCAR), an international organization that coordinates scientific exploration and environmental protection in the region.

### 2. Mapping Antarctica's Research Stations 🔍

For this challenge, we'll be focusing on mapping the research stations and bases located throughout Antarctica. These facilities, both permanent and temporary, are operated by various national programs and serve as hubs for scientific research, exploration, and logistical support.

The data for this map comes from the SCAR database, which maintains a comprehensive list of all research stations and their locations. We'll be using this information to create a detailed visual representation of the Antarctic research landscape.

### 3. Data Preparation and Processing 💻

First, let's import the necessary libraries and load the SCAR data into a pandas DataFrame:

In [1]:
# Install the Folium library for interactive maps
!pip install folium -q

In [2]:
# Import required libraries
import pandas as pd
import re
import folium
from folium.plugins import MousePosition
from folium import WmsTileLayer

In [3]:
# Create a destination folder to store datasets
!mkdir -p datasets

# Download the specific CSV file from GitHub and save it in the 'datasets' folder
!wget -O datasets/stations_bases_aq.csv https://raw.githubusercontent.com/oechenique/30DayMapChallenge/960fc125666dce4a39075c7b02f632c346ea4692/Notebooks/data/data_day11/stations_bases_aq.csv

--2024-11-12 18:55:11--  https://raw.githubusercontent.com/oechenique/30DayMapChallenge/960fc125666dce4a39075c7b02f632c346ea4692/Notebooks/data/data_day11/stations_bases_aq.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5150 (5.0K) [text/plain]
Saving to: ‘datasets/stations_bases_aq.csv’

datasets/stations_b 100%[===================>]   5.03K  --.-KB/s    in 0s      

2024-11-12 18:55:12 (35.4 MB/s) - ‘datasets/stations_bases_aq.csv’ saved [5150/5150]



In [4]:
# Load the CSV file into a DataFrame called 'df'
df = pd.read_csv('/content/datasets/stations_bases_aq.csv')

Next, we'll clean and process the data to prepare it for mapping:

In [5]:
# Function to convert from DMS (Degrees, Minutes, Seconds) format to decimal degrees
def dms_to_decimal(dms):
    # Regular expression to capture degrees, optional minutes, optional seconds, and direction
    match = re.match(r"(\d+)[^\d]+(\d+)?[^\d]*?(\d+)?[^\d]*([NSEW])", dms)
    if not match:
        raise ValueError(f"Invalid DMS format: {dms}")

    # Extract the degree, minute, second, and direction components
    degrees = float(match.group(1))
    minutes = float(match.group(2)) if match.group(2) else 0
    seconds = float(match.group(3)) if match.group(3) else 0
    direction = match.group(4)

    # Convert to decimal degrees
    decimal = degrees + minutes / 60 + seconds / 3600
    # Make negative if the direction is 'S' (South) or 'W' (West)
    if direction in ['S', 'W']:
        decimal = -decimal
    return decimal

# Function to apply the conversion function to each row in the DataFrame
def convert_coordinates(df):
    df['latitude_decimal'] = df['latitude'].apply(dms_to_decimal)  # Convert latitude to decimal format
    df['longitude_decimal'] = df['longitude'].apply(dms_to_decimal)  # Convert longitude to decimal format
    return df

# Assuming your DataFrame is named 'df'
df = convert_coordinates(df)

Now, let's create the map and add the research station markers:

In [6]:
# Create a map centered on Antarctica without zoom controls
m = folium.Map(
    location=[-75, 0],  # Center the map on Antarctica
    tiles=None,  # Disable default background tiles
    zoom_start=3,
    max_bounds=True,
    control_scale=False,  # Disable zoom controls
    dragging=True  # Allow zoom with scroll without showing controls
)

# Add the GEBCO 2024 bathymetry layer using WMS (Web Map Service)
wms_layer = WmsTileLayer(
    url="https://wms.gebco.net/2024/mapserv?",
    layers="GEBCO_2024",
    fmt="image/png",
    transparent=True
)
wms_layer.add_to(m)

# Define color mapping for facility types
colors = {'year-round': 'yellow', 'seasonal': 'red'}

# Add Antarctic bases with additional information in the popup
for idx, row in df.iterrows():
    folium.CircleMarker(
        location=(row['latitude_decimal'], row['longitude_decimal']),
        radius=5,
        color=colors[row['facility']],
        fill=True,
        fill_color=colors[row['facility']],
        fill_opacity=0.8,
        popup=folium.Popup(
            f"<strong>Base Name:</strong> {row['name']}<br>"
            f"<strong>Country:</strong> {row['country']}<br>"
            f"<strong>Facility:</strong> {row['facility']}",
            max_width=300  # Set maximum width for the popup
        ),
    ).add_to(m)

# Add a styled map title
title_html = '''
    <div style="position: fixed; top: 10px; width: 100%; text-align: center;
                font-size: 24px; font-weight: bold; color: white; background-color: #1E90FF; padding: 10px; z-index: 9999;">
        🌏 Antarctic Bases Information with GEBCO Basemap 🗻
    </div>
'''
m.get_root().html.add_child(folium.Element(title_html))

# Add a legend with consistent styling
legend_html = '''
     <div style="
     position: fixed;
     bottom: 50px; left: 10px; width: 112px; height: 90px;
     background-color: #1E90FF; border:2px solid white; z-index:9999; font-size:14px;
     color: white; padding: 10px; border-radius: 5px;">
     <b>Legend</b><br>
     <i class="fa fa-circle" style="color:yellow"></i> Year-round<br>
     <i class="fa fa-circle" style="color:red"></i> Seasonal
     </div>
     '''
m.get_root().html.add_child(folium.Element(legend_html))

# Display the interactive map
m

In [7]:
# Save the map as an HTML file
m.save("antarctic_bases_map.html")

In [8]:
# Open the HTML file containing the map
with open("antarctic_bases_map.html", "r") as file:
    map_html = file.read()

# Add responsive styling to the map container
map_html = map_html.replace(
    '<div style="width:100%;height:100%;">',
    '<div style="width:100%;height:100vh;max-width:100%;overflow:hidden;">'
)

# Save the modified HTML file
with open("antarctic_bases_map.html", "w") as file:
    file.write(map_html)

This code sets up a base map centered on Antarctica, creates a marker cluster to group the research stations, and then adds each station as a marker with its name, nation, and type displayed in the popup.


### 5. The Nippon Foundation and GEBCO 🇯🇵

It's worth noting that the bathymetric data for this map comes from the GEBCO (General Bathymetric Chart of the Oceans) project, which is supported by the Nippon Foundation, a Japanese non-profit organization.

The Nippon Foundation has been a long-time supporter of GEBCO's efforts to map the world's oceans and seafloors, including the waters surrounding Antarctica. This collaboration has been crucial in advancing our understanding of the Antarctic region and its unique geological features.

### 6. Exploring Antarctica's Frozen Wonders 🧊

With this map, we can visualize the distribution of research stations across the Antarctic continent, gaining insights into the international collaboration and scientific activities taking place in this remote and inhospitable region.

By highlighting the permanent and seasonal stations operated by various nations, we can better understand the logistical challenges and the importance of these facilities in furthering our knowledge of the Antarctic environment, climate, and ecosystems.

### 7. Additional Resources 📚

- [SCAR - Scientific Committee on Antarctic Research](https://www.scar.org/)
- [Antarctic Treaty System](https://www.ats.aq/)
- [GEBCO - General Bathymetric Chart of the Oceans](https://www.gebco.net/)
- [The Nippon Foundation](https://www.nippon-foundation.or.jp/en/)

### 8. Safety Considerations ⚠️

Remember, the Antarctic continent is a harsh and unforgiving environment. Only experienced and trained personnel should attempt to visit or work in this region. For the safety of all, it's crucial to follow the guidelines and regulations set forth by the Antarctic Treaty System.

### 9. Arigatou Gozaimasu! 🙏

Thank you for exploring the frozen wonders of Antarctica with me today. I hope this map has provided a glimpse into the scientific and geopolitical significance of this unique continent. Let's continue our journey through the #30DayMapChallenge, one map at a time!

In [9]:
#@title Follow Me!
%%html

<div style="background: linear-gradient(90deg, #1a1a1a, #2d2d2d); color: white; padding: 2rem; border-radius: 10px; font-family: 'Roboto', sans-serif; margin-top: 2rem;">
    <h2 style="text-align: center; margin-bottom: 1rem;">
        <span style="display: block; color: white; font-size: 2.7rem; margin-bottom: 0.5rem;">ありがとうございます！</span>
        <span style="display: block; color: white; font-size: 1.8rem;">🌍 Thank you for making it this far! 🎌</span>
    </h2>

    <p style="font-size: 1.2rem; margin-bottom: 1.5rem;">If you're interested in the world of geoinformatics and want to keep learning, connect with me! | 一緒に学びましょう！</p>

    <div style="display: flex; flex-wrap: wrap; gap: 1rem; margin-bottom: 1rem;">
        <a href="https://x.com/GastonEchenique" target="_blank" style="text-decoration: none; color: white; display: flex; align-items: center; background: #1DA1F2; padding: 0.5rem 1rem; border-radius: 5px; transition: transform 0.2s;">
            <svg style="width: 24px; height: 24px; margin-right: 0.5rem;" viewBox="0 0 24 24" fill="currentColor">
                <path d="M18.244 2.25h3.308l-7.227 8.26 8.502 11.24H16.17l-5.214-6.817L4.99 21.75H1.68l7.73-8.835L1.254 2.25H8.08l4.713 6.231zm-1.161 17.52h1.833L7.084 4.126H5.117z"/>
            </svg>
            <span>@GastonEchenique</span>
        </a>

        <a href="https://www.linkedin.com/in/gaston-echenique/" target="_blank" style="text-decoration: none; color: white; display: flex; align-items: center; background: #0A66C2; padding: 0.5rem 1rem; border-radius: 5px; transition: transform 0.2s;">
            <svg style="width: 24px; height: 24px; margin-right: 0.5rem;" viewBox="0 0 24 24" fill="currentColor">
                <path d="M20.447 20.452h-3.554v-5.569c0-1.328-.027-3.037-1.852-3.037-1.853 0-2.136 1.445-2.136 2.939v5.667H9.351V9h3.414v1.561h.046c.477-.9 1.637-1.85 3.37-1.85 3.601 0 4.267 2.37 4.267 5.455v6.286zM5.337 7.433c-1.144 0-2.063-.926-2.063-2.065 0-1.138.92-2.063 2.063-2.063 1.14 0 2.064.925 2.064 2.063 0 1.139-.925 2.065-2.064 2.065zm1.782 13.019H3.555V9h3.564v11.452zM22.225 0H1.771C.792 0 0 .774 0 1.729v20.542C0 23.227.792 24 1.771 24h20.451C23.2 24 24 23.227 24 22.271V1.729C24 .774 23.2 0 22.222 0h.003z"/>
            </svg>
            <span>Gastón Echenique</span>
        </a>

        <a href="https://github.com/oechenique" target="_blank" style="text-decoration: none; color: white; display: flex; align-items: center; background: #333; padding: 0.5rem 1rem; border-radius: 5px; transition: transform 0.2s;">
            <svg style="width: 24px; height: 24px; margin-right: 0.5rem;" viewBox="0 0 24 24" fill="currentColor">
                <path d="M12 0c-6.626 0-12 5.373-12 12 0 5.302 3.438 9.8 8.207 11.387.599.111.793-.261.793-.577v-2.234c-3.338.726-4.033-1.416-4.033-1.416-.546-1.387-1.333-1.756-1.333-1.756-1.089-.745.083-.729.083-.729 1.205.084 1.839 1.237 1.839 1.237 1.07 1.834 2.807 1.304 3.492.997.107-.775.418-1.305.762-1.604-2.665-.305-5.467-1.334-5.467-5.931 0-1.311.469-2.381 1.236-3.221-.124-.303-.535-1.524.117-3.176 0 0 1.008-.322 3.301 1.23.957-.266 1.983-.399 3.003-.404 1.02.005 2.047.138 3.006.404 2.291-1.552 3.297-1.23 3.297-1.23.653 1.653.242 2.874.118 3.176.77.84 1.235 1.911 1.235 3.221 0 4.609-2.807 5.624-5.479 5.921.43.372.823 1.102.823 2.222v3.293c0 .319.192.694.801.576 4.765-1.589 8.199-6.086 8.199-11.386 0-6.627-5.373-12-12-12z"/>
            </svg>
            <span>oechenique</span>
        </a>

        <a href="https://oechenique.github.io/geoanalytics/" target="_blank" style="text-decoration: none; color: white; display: flex; align-items: center; background: #2ecc71; padding: 0.5rem 1rem; border-radius: 5px; transition: transform 0.2s;">
            <svg style="width: 24px; height: 24px; margin-right: 0.5rem;" viewBox="0 0 24 24" fill="currentColor">
                <path d="M12 0c-6.627 0-12 5.373-12 12s5.373 12 12 12 12-5.373 12-12-5.373-12-12-12zm1 16.057v-3.057h2.994c-.059 1.143-.212 2.24-.456 3.279-.823-.12-1.674-.188-2.538-.222zm1.957 2.162c-.499 1.33-1.159 2.497-1.957 3.456v-3.62c.666.028 1.319.081 1.957.164zm-1.957-7.219v-3.015c.868-.034 1.721-.103 2.548-.224.238 1.027.389 2.111.446 3.239h-2.994zm0-5.014v-3.661c.806.969 1.471 2.15 1.971 3.496-.642.084-1.3.137-1.971.165zm2.703-3.267c1.237.496 2.354 1.228 3.29 2.146-.642.234-1.311.442-2.019.607-.344-.992-.775-1.91-1.271-2.753zm-7.241 13.56c-.244-1.039-.398-2.136-.456-3.279h2.994v3.057c-.865.034-1.714.102-2.538.222zm2.538 1.776v3.62c-.798-.959-1.458-2.126-1.957-3.456.638-.083 1.291-.136 1.957-.164zm-2.994-7.055c.057-1.128.207-2.212.446-3.239.827.121 1.68.19 2.548.224v3.015h-2.994zm1.024-5.179c.5-1.346 1.165-2.527 1.97-3.496v3.661c-.671-.028-1.329-.081-1.97-.165zm-2.005-.35c-.708-.165-1.377-.373-2.018-.607.937-.918 2.053-1.65 3.29-2.146-.496.844-.927 1.762-1.272 2.753zm-.549 1.918c-.264 1.151-.434 2.36-.492 3.611h-3.933c.165-1.658.739-3.197 1.617-4.518.88.361 1.816.67 2.808.907zm.009 9.262c-.988.236-1.92.542-2.797.9-.89-1.328-1.471-2.879-1.637-4.551h3.934c.058 1.265.231 2.488.5 3.651zm.553 1.917c.342.976.768 1.881 1.257 2.712-1.223-.49-2.326-1.211-3.256-2.115.636-.229 1.299-.435 1.999-.597zm9.924 0c.7.163 1.362.367 1.999.597-.931.903-2.034 1.625-3.257 2.116.489-.832.915-1.737 1.258-2.713zm.553-1.917c.27-1.163.442-2.386.501-3.651h3.934c-.167 1.672-.748 3.223-1.638 4.551-.877-.358-1.81-.664-2.797-.9zm.501-5.651c-.058-1.251-.229-2.46-.492-3.611.992-.237 1.929-.546 2.809-.907.877 1.321 1.451 2.86 1.616 4.518h-3.933z"/>
            </svg>
            <span>GeoAnalytics</span>
        </a>

        <a href="https://discord.com/users/gastonechenique" target="_blank" style="text-decoration: none; color: white; display: flex; align-items: center; background: #5865F2; padding: 0.5rem 1rem; border-radius: 5px; transition: transform 0.2s;">
            <svg style="width: 24px; height: 24px; margin-right: 0.5rem;" viewBox="0 0 24 24" fill="currentColor">
                <path d="M20.317 4.37a19.791 19.791 0 0 0-4.885-1.515a.074.074 0 0 0-.079.037c-.21.375-.444.864-.608 1.25a18.27 18.27 0 0 0-5.487 0a12.64 12.64 0 0 0-.617-1.25a.077.077 0 0 0-.079-.037A19.736 19.736 0 0 0 3.677 4.37a.07.07 0 0 0-.032.027C.533 9.046-.32 13.58.099 18.057a.082.082 0 0 0 .031.057a19.9 19.9 0 0 0 5.993 3.03a.078.078 0 0 0 .084-.028a14.09 14.09 0 0 0 1.226-1.994a.076.076 0 0 0-.041-.106a13.107 13.107 0 0 1-1.872-.892a.077.077 0 0 1-.008-.128a10.2 10.2 0 0 0 .372-.292a.074.074 0 0 1 .077-.01c3.928 1.793 8.18 1.793 12.062 0a.074.074 0 0 1 .078.01c.12.098.246.198.373.292a.077.077 0 0 1-.006.127a12.299 12.299 0 0 1-1.873.892a.077.077 0 0 0-.041.107c.36.698.772 1.362 1.225 1.993a.076.076 0 0 0 .084.028a19.839 19.839 0 0 0 6.002-3.03a.077.077 0 0 0 .032-.054c.5-5.177-.838-9.674-3.549-13.66a.061.061 0 0 0-.031-.03zM8.02 15.33c-1.183 0-2.157-1.085-2.157-2.419c0-1.333.956-2.419 2.157-2.419c1.21 0 2.176 1.096 2.157 2.42c0 1.333-.956 2.418-2.157 2.418zm7.975 0c-1.183 0-2.157-1.085-2.157-2.419c0-1.333.955-2.419 2.157-2.419c1.21 0 2.176 1.096 2.157 2.42c0 1.333-.946 2.418-2.157 2.418z"/>
            </svg>
            <span>Gaston | ガストン</span>
        </a>
    </div>

    <p style="font-size: 1rem; color: #888;">💡 頑張りましょう！Don't miss more content about GIS, Data Science and Geoinformatics!</p>

    <div style="margin-top: 1rem; padding-top: 1rem; border-top: 1px solid #444; font-size: 0.9rem; color: #666;">
        Made with 🗺️ & 💻 by Gaston Echenique | エチェニケ・ガストン
    </div>
</div>